In [1]:
from source.regency import Regency
from source.DQNAgent import DQNAgent
from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')


seasons = 1
Game = Regency(IntDC=30)
clear_output()

for a in range(seasons):
    print(a)
    Game.random_events()
    Game.collect_regency_points()
    Game.domain_initiative()
    Game.collect_gold_bars()
    Game.maintenance_costs(Update=True)
    Game.take_domain_actions(None)
Game.clear_screen()
Game.Seasons[Game.Season]['Season']

,Regent,Player,Random Event,Regency Points,Initiative,Gold Bars,Revenue,Cost,Court
0,Ch,False,No Event,28,23,0,0,0,Dormant
1,GH,False,Trade Matter,10,20,4,7,3,Bare
2,Sw,False,No Event,11,20,0,0,0,Dormant
3,LPA,False,Natural Event,11,20,5,9,4,Bare
4,WM,False,Feud,7,19,1,2,1,Dormant
5,WB,False,No Event,2,18,1,2,1,Dormant
6,DW,False,No Event,71,18,33,11,11,Rich
7,MF,False,Brigandage,59,16,25,6,9,Rich
8,IHH,False,No Event,6,14,2,3,1,Dormant
9,CSH,False,Diplomatic Matter,10,14,4,7,3,Bare


In [2]:
import pandas as pd
df = pd.concat([pd.concat([Game.Seasons[a]['Actions'][b+1] for b in range(3)]) for a in range(seasons)])

In [3]:
Game.Troops.head()

,CR,Cost,Garrisoned,Home,Injury,Provence,Regent,Type
0,3.0,2,0,,0,Moergen,JR,Cavalry
1,3.0,2,0,,0,Moergen,JR,Cavalry
2,1.0,1,0,,0,Moergen,JR,Archers
3,1.0,1,0,,0,Moergen,JR,Pikemen
5,3.0,2,0,,0,Bogsend,EC,Cavalry


In [4]:
set(df['Action'])

{'agitiate_for_friend',
 'declare_war',
 'decree_asset_seizure',
 'decree_general',
 'diplomacy_trade_agreement',
 'disband_army',
 'disband_mercenaries',
 'espionage_diplomatic_details',
 'espionage_discover_troop_movements',
 'espionage_heresy',
 'espionage_intrigue',
 'grant',
 'muster_army',
 'muster_mercenaries'}

In [5]:
import pandas as pd
import numpy as np
self = Game
self.Troops

,CR,Cost,Garrisoned,Home,Injury,Provence,Regent,Type
0,3.0,2,0,,0,Moergen,JR,Cavalry
1,3.0,2,0,,0,Moergen,JR,Cavalry
2,1.0,1,0,,0,Moergen,JR,Archers
3,1.0,1,0,,0,Moergen,JR,Pikemen
5,3.0,2,0,,0,Bogsend,EC,Cavalry
6,3.0,2,0,,0,Bogsend,EC,Cavalry
7,1.0,1,0,,0,Bogsend,EC,Archers
8,1.0,1,0,,0,Bogsend,EC,Pikemen
9,2.0,2,0,,0,Rivenrock,GG,Dwarf Guards
10,2.0,2,0,,0,Rivenrock,GG,Dwarf Crossbows


In [151]:
temp = self.Provences.copy()
temp['Home Regent'] = temp['Regent']
temp = pd.merge(self.Troops, temp[['Home Regent', 'Provence']], on='Provence', how='left')

lst = []
temp['Defense'] = 0
temp['Offense'] = 0
M = temp[temp['Regent'] == temp['Home Regent']]
M['Defense'] = 1
lst.append(M)
temp =  temp[temp['Regent'] != temp['Home Regent']]
for HR in set(temp['Home Regent']):
    allies, enemies = self.allies_enemies(HR)
    
    E = pd.merge(enemies, temp[temp['Home Regent']==HR], on='Regent', how='left').fillna(0)
    E['Offense'] = 1
    A = pd.merge(allies, temp[temp['Home Regent']==HR], on='Regent', how='left').fillna(0)
    A['Defense'] = 1
    lst.append(E)
    lst.append(A)
temp = pd.concat(lst)
temp =  temp[temp['Provence'] !=0]

defenders = temp[temp['Defense']==1]
attackers = temp[temp['Offense']==1]

attackers

,CR,Cost,Defense,Garrisoned,Home,Home Regent,Injury,Offense,Provence,Regent,Type
1,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
2,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
3,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
4,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
5,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
6,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
7,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers
8,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers


In [152]:
Provence = 'Moergen'

offense = attackers[attackers['Provence'] == Provence]
defense =  defenders[defenders['Provence'] == Provence]

offense['mod'] = offense['Injury']
defense['mod'] = defense['Injury']
# Enemy force has Archer-class units and your force has no Cavalry-class units    -1
if np.sum(offense['Type'].str.lower().str.contains('archer')) >=1 and\
   np.sum(defense['Type'].str.lower().str.contains('cavalry|knight')) == 0:
    defense['mod'] = defense['mod'] - 1
if np.sum(defense['Type'].str.lower().str.contains('archer')) >=1 and\
   np.sum(offense['Type'].str.lower().str.contains('cavalry|knight')) == 0:
    offense['mod'] = offense['mod'] - 1
# Enemy force has Cavalry-class units and your force has no Pikemen or Cavalry-class units    -1
if np.sum(offense['Type'].str.lower().str.contains('cavalry|knight')) >=1 and\
   np.sum(defense['Type'].str.lower().str.contains('cavalry|knight|pikem')) == 0:
    defense['mod'] = defense['mod'] - 1
if np.sum(defense['Type'].str.lower().str.contains('archer')) >=1 and\
   np.sum(offense['Type'].str.lower().str.contains('cavalry|knight|pikem')) == 0:
    offense['mod'] = offense['mod'] - 1
# Per 2 total BCR the enemy force exceeds your own (maximum penalty -3)   -1
# Per 2 total BCR your force exceeds the enemy force (maximum bonus +3)   +1
for a in range(0,int((np.sum(defense['CR'])-np.sum(offense['CR']))/2)):
    if a <= 2:
        offense['mod'] = offense['mod']-1
        defense['mod'] = defense['mod']+1
for a in range(0,int((np.sum(offense['CR'])-np.sum(defense['CR']))/2)):
    if a <= 2:
        offense['mod'] = offense['mod']+1
        defense['mod'] = defense['mod']-1
# The unit has terrain advantage (elves in forest, dwarves in mountains)  +1 [Skipping for now]
# Your force has established fortifications and defenses and enemy force has no Artillery or Engineer-class units.    +1
if self.Provences[self.Provences['Provence']==Provence]['Castle'].values[0] > 0 and np.sum(offense['Type'].str.lower().str.contains('artillery|engineer')) == 0:
        defense['mod'] = defense['mod'] + 1
# The unit possesses an attached commander.   +1  [May Add later]
offense

,CR,Cost,Defense,Garrisoned,Home,Home Regent,Injury,Offense,Provence,Regent,Type,mod
1,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
2,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
3,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
4,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
5,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
6,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
7,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2
8,1.0,1,0.0,0,,JR,0,1.0,Moergen,WM,Archers,-2


In [163]:
off_score = 0
def_score = 0

while off_score == def_score:
    offense['roll'] = np.random.randint(1,6,offense.shape[0]) + offense['mod']
    defense['roll'] = np.random.randint(1,6,defense.shape[0]) + defense['mod']
    off_score = np.sum(defense['roll']<=1)
    def_score = np.sum(offense['roll']<=1)
    # deal with causulties
    offense_50 = offense[offense['roll']==1]
    defense_50 = defense[defense['roll']==1]
    offense_25 = offense[offense['roll']>=2]
    defense_25 = defense[defense['roll']>=2]
    offense_25 = offense_25[offense_25['roll']<=5]
    defense_25 = defense_25[defense_25['roll']<=5]
    offense_good = offense[offense['roll']>5]
    defense_good = defense[defense['roll']>5]
    offense_50['Injury'] = offense_50['Injury'] - 2
    defense_50['Injury'] = defense_50['Injury'] - 2
    offense_25['Injury'] = offense_25['Injury'] - 1
    defense_25['Injury'] = defense_25['Injury'] - 1
    dead = pd.concat([offense[offense['roll']<=0], defense[defense['roll']<=0]])
    offense = pd.concat([offense_50,offense_25,offense_good])
    defense = pd.concat([defense_50,defense_25,defense_good])
    dead = pd.concat([dead, offense[offense['Injury']<=-4], defense[defense['Injury']<=-4]])
    for i, row in dead.iterrows():
        self.disband_troops(row['Regent'], row['Provence'], row['Type'], True)  # kill 'em
# run away if alive     
retreat = offense
if def_score > off_score:
    retreat = defense
for i, row in retreat.i

In [168]:
escape = pd.merge(self.Geography[self.Geography['Provence']==Provence],self.Troops[self.Troops['Regent']],on=Provence
esca

,Provence,Neighbor,Border,Road,Caravan,Shipping,RiverChasm
28,Moergen,Ghalliere,1,0,0,0,0
30,Moergen,Gulfport,1,0,0,0,0
39,Moergen,Moriel,1,0,0,0,0
51,Moergen,Sunken Lands,1,0,0,0,1
55,Moergen,Bogsend,1,0,0,0,1


In [166]:
defense

,CR,Cost,Defense,Garrisoned,Home,Home Regent,Injury,Offense,Provence,Regent,Type,mod,roll
0,3.0,2,1.0,0,,JR,-5,0.0,Moergen,JR,Cavalry,0,1
1,3.0,2,1.0,0,,JR,-4,0.0,Moergen,JR,Cavalry,0,1
2,1.0,1,1.0,0,,JR,-4,0.0,Moergen,JR,Archers,0,5
3,1.0,1,1.0,0,,JR,-3,0.0,Moergen,JR,Pikemen,0,3


In [167]:
offense

,CR,Cost,Defense,Garrisoned,Home,Home Regent,Injury,Offense,Provence,Regent,Type,mod,roll
2,1.0,1,0.0,0,,JR,-4,1.0,Moergen,WM,Archers,-2,3
6,1.0,1,0.0,0,,JR,-4,1.0,Moergen,WM,Archers,-2,2
8,1.0,1,0.0,0,,JR,-3,1.0,Moergen,WM,Archers,-2,3


In [142]:
self.Troops

,CR,Cost,Garrisoned,Home,Injury,Provence,Regent,Type
0,3.0,2,0,,0,Moergen,JR,Cavalry
1,3.0,2,0,,0,Moergen,JR,Cavalry
2,1.0,1,0,,0,Moergen,JR,Archers
3,1.0,1,0,,0,Moergen,JR,Pikemen
4,3.0,2,0,,0,Bogsend,EC,Cavalry
5,3.0,2,0,,0,Bogsend,EC,Cavalry
6,1.0,1,0,,0,Bogsend,EC,Archers
7,1.0,1,0,,0,Bogsend,EC,Pikemen
8,2.0,2,0,,0,Rivenrock,GG,Dwarf Guards
9,2.0,2,0,,0,Rivenrock,GG,Dwarf Crossbows
